In [ ]:
demrst = '/home/osmtolulc/mrgis/vistofire/cmb_dem10.tif'
pntobs = '/home/osmtolulc/mrgis/vistofire/pnt_incendio.shp'
obs_id = 'pnt_fid'

In [ ]:
db = 'viewtofire_1'

pnt_id = [1, 2]

In [ ]:
from osgeo                 import gdal
from glass.sql.fm import q_to_obj
import numpy as np
from glass.it.rst import obj_to_rst

In [ ]:
# Get Raster Shape

rsrc = gdal.Open(demrst, gdal.GA_ReadOnly)

rows = rsrc.RasterYSize
cols = rsrc.RasterXSize

In [ ]:
# Get Array with cells with visibility == 1

Q = (
    "SELECT idx FROM ("
        "SELECT generate_series("
            "(rowi * {c}) + coli, "
            "(rowe * {c}) + cole "
        ") AS idx "
        "FROM vistoburn "
        "WHERE {whr}"
    ") AS foo "
    "GROUP BY idx "
    "ORDER BY idx"
).format(
    c=str(cols), whr=" OR ".join([
        "pntid = {}".format(str(p)) for p in pnt_id
    ])
)

visidx = q_to_obj(db, Q, db_api='psql')

In [ ]:
allidx = np.arange(rows * cols).astype(np.uint32)
visimg = np.zeros(rows * cols)

In [ ]:
np.place(visimg, np.isin(allidx, visidx.idx), 1)

In [ ]:
visimg = visimg.reshape((rows, cols))

obj_to_rst(visimg, '/home/osmtolulc/mrgis/vistofire/teste_vis12.tif', rsrc, noData=0)

In [ ]:
print(np.unique(visimg))

In [ ]:
import os
import pandas              as pd
import numpy               as np

import multiprocessing     as mp
from glass.dct.geo.fmshp    import shp_to_obj
from glass.pys.oss         import cpu_cores, mkdir
from glass.pd.split     import df_split
from glass.wenv.grs    import run_grass
from glass.prop.prj import shp_epsg
from glass.sql.to          import df_to_db
from glass.pys.oss         import del_file
from glass.sql.db          import create_db
from glass.pys.num         import get_minmax_fm_seq_values

In [ ]:
rst = '/home/osmtolulc/mrgis/vistofire/work_2/vistoburn_2650.tif'

In [ ]:
img = gdal.Open(rst, gdal.GA_ReadOnly)

num = img.ReadAsArray()

numone = num.reshape(num.shape[0] * num.shape[1])

visnum = np.arange(numone.shape[0]).astype(np.uint32)

visnum = visnum[numone == 1]

In [ ]:
print(visnum.shape)

In [ ]:
visint = get_minmax_fm_seq_values(visnum)

In [ ]:
print(visint.shape)

In [ ]:
_visint = visint.reshape(visint.shape[0] * visint.shape[1])
print(_visint.shape)

In [ ]:
print(_visint)

In [ ]:
visrow = _visint / num.shape[1]
visrow = visrow.astype(np.uint32)
print(visrow.shape)

In [ ]:
viscol = _visint - (visrow * num.shape[1])

print(viscol.shape)

In [ ]:
visrow = visrow.reshape(visint.shape)
viscol = viscol.reshape(visint.shape)

print(visrow.shape)

In [ ]:
irow, erow = np.vsplit(visrow.T, 1)[0]
icol, ecol = np.vsplit(viscol.T, 1)[0]

print(irow.shape)

In [ ]:
print(erow.shape)

In [ ]:
visdf = pd.DataFrame({
    'rowi' : irow, 'rowe' : erow,
    'coli' : icol, 'cole' : ecol
})

In [ ]:
idxnum = np.full(visrow.shape, 1)

print(idxnum.shape)